This notebook is meant to run on WSL2, hence the directml usage.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import nrrd
import torch
import torch.nn as nn

from PIL import Image
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from torch.utils.data import Dataset, DataLoader
import torch_directml as tdml

print(f'Available devices: {tdml.device_count()}')
print(f'Current device: {tdml.device()}')
dml = tdml.device()

In [ ]:
class RasterDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir

        raw_labels = [(*(os.path.splitext(file)[0].split("_")[1:-1]), file) for file in os.listdir(data_dir)]

        self.labels = pd.DataFrame(data=raw_labels, columns=["pid", "age", "tbv", "filename"])
        self.labels[["age", "tbv"]] = self.labels[["age", "tbv"]].astype(float)

        self.age_minmax = MinMaxScaler()
        self.tbv_minmax = MinMaxScaler()
        self.age_std = StandardScaler()
        self.tbv_std = StandardScaler()

        self.labels[["age"]] = self.age_minmax.fit_transform(self.labels[["age"]])
        self.labels[["tbv"]] = self.tbv_minmax.fit_transform(self.labels[["tbv"]])
        self.labels[["age"]] = self.age_std.fit_transform(self.labels[["age"]])
        self.labels[["tbv"]] = self.tbv_std.fit_transform(self.labels[["tbv"]])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        entry = self.labels.iloc[idx]
        raster = nrrd.read(os.path.join(self.data_dir, entry["filename"]))[0]
        raster = (raster - np.mean(raster)) / np.std(raster) # Standardize the data

        return {"pid": entry["pid"], "age": entry["age"], "tbv": entry["tbv"], "raster": raster}

    def get_unique_pids(self):
        return self.labels["pid"].unique()
    
    def get_indices_from_pids(self, pids):
        return self.labels[self.labels["pid"].isin(pids)].index
        
class RasterNet(nn.Module):
    def __init__(self):
        super(RasterNet, self).__init__()

        # First layer is triplanar as used in S3PNet:
        # https://www.sciencedirect.com/science/article/pii/S1077314219301791

        self.trpl = nn.Sequential( # 128x128x128 -> 32x40x40
            nn.Conv2d(128, 32, kernel_size=9, stride=3, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )

        self.conv1 = nn.Sequential( # 96x40x40 -> 192x10x10
            nn.Conv2d(96, 192, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv2 = nn.Sequential( # 192x10x10 -> 384x5x5
            nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv3 = nn.Sequential( # 384x5x5 -> 768x3x3
            nn.Conv2d(384, 768, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(768),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv4 = nn.Sequential( # 768x3x3 -> 1536x1x1
            nn.Conv2d(768, 1536, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1536),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.linear = nn.Sequential(
            nn.Linear(1536, 768),
            nn.ReLU(),
            nn.Linear(768, 1)
        )

    def forward(self, base):
        base = base.squeeze().float().to(dml)

        xy = self.trpl(base)
        yz = self.trpl(torch.transpose(base, 2, 1))
        xz = self.trpl(torch.transpose(base, 1, 3))

        x = self.conv1(torch.cat((xy, yz, xz), dim=1))
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return self.linear(x.squeeze())

class EarlyStopper:
    def __init__(self, patience=25, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model=None):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            if model: self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            if model: self.save_checkpoint(val_loss, model)
            self.counter = 0
        
        return self.early_stop

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
def train(model, criterion, optimizer, tr_dataloader, val_dataloader, early_stopper, num_epochs, trace_func=print):
    train_loss_list = pd.Series(dtype=np.float32)
    val_loss_list = pd.Series(dtype=np.float32)

    for epoch in range(num_epochs):
        training_loss = 0.
        
        model.train()
        for i, data in enumerate(tr_dataloader):
            rasters = data["raster"].float().unsqueeze(1).to(dml)
            tbvs = data["tbv"].float().to(dml)

            optimizer.zero_grad()

            predictions = model(rasters).squeeze()
            loss = criterion(predictions, tbvs)

            loss.backward()
            optimizer.step()

            training_loss += loss.item()

            if (i+1) % 10 == 0:
                trace_func(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(tr_dataloader)}], Loss: {loss.item():.4f}              ", end="\r")

        validation_loss = 0.
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(val_dataloader):
                rasters = data["raster"].float().unsqueeze(1).to(dml)
                tbvs = data["tbv"].float().to(dml)

                predictions = model(rasters).squeeze()
                loss = criterion(predictions, tbvs)

                validation_loss += loss.item()

        train_loss = training_loss/len(tr_dataloader)
        val_loss = validation_loss/len(val_dataloader)
        train_loss_list.at[epoch] = train_loss
        val_loss_list.at[epoch] = val_loss

        trace_func(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}               ")
        if early_stopper(validation_loss, model):
            break
    
    return train_loss_list, val_loss_list

def get_unscaled_loss(model, criterion, dataloader, dataset):
    all_diffs = []
    for i, data in enumerate(dataloader):
        rasters = data["raster"].float().unsqueeze(1)
        tbvs = data["tbv"].float()

        predictions = model(rasters).squeeze().cpu()
        predictions = predictions.detach().numpy().reshape(-1, 1)
        tbvs = tbvs.detach().numpy().reshape(-1, 1)
        
        # Revert the normalization and standardization
        predictions = dataset.tbv_std.inverse_transform(predictions)
        predictions = dataset.tbv_minmax.inverse_transform(predictions)
        tbvs = dataset.tbv_std.inverse_transform(tbvs)
        tbvs = dataset.tbv_minmax.inverse_transform(tbvs)

        all_diffs.append(np.abs(predictions - tbvs))

    all_diffs = np.concatenate(all_diffs)
    
    avg_diff = np.mean(all_diffs)
    std_diff = np.std(all_diffs)

    return avg_diff, std_diff

def cross_validator(model, dataset, k_fold=5, num_epochs=100, patience=25, learning_rate=0.001, verbose=False, trace_func=print):
    # Save model to reset it after each fold
    torch.save(model.state_dict(), "base_weights.pt")

    train_score = pd.Series(dtype=np.float32)
    val_score = pd.Series(dtype=np.float32)
    unscaled_loss = pd.Series(dtype=np.float32)
    
    unique_pids = dataset.get_unique_pids()
    total_pids = len(unique_pids)
    fraction = 1/k_fold
    seg = int(total_pids*fraction)
    
    # tr:train, val:valid; r:right,l:left;  eg: trrr: right index of right side train subset 
    # index: [trll,trlr],[vall,valr],[trrl,trrr]
    for i in range(k_fold):
        trace_func(f"Fold {i+1}/{k_fold}")

        model.load_state_dict(torch.load("base_weights.pt"))
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        early_stopper = EarlyStopper(patience=patience, verbose=verbose, path="best_weights.pt", trace_func=trace_func)

        trll = 0
        trlr = i * seg
        vall = trlr
        valr = i * seg + seg
        trrl = valr
        trrr = total_pids
        
        train_left_indices = list(range(trll,trlr))
        train_right_indices = list(range(trrl,trrr))
        
        train_indices = train_left_indices + train_right_indices
        val_indices = list(range(vall,valr))

        train_indices = dataset.get_indices_from_pids(unique_pids[train_indices])
        val_indices = dataset.get_indices_from_pids(unique_pids[val_indices])

        train_set = torch.utils.data.dataset.Subset(dataset, train_indices)
        val_set = torch.utils.data.dataset.Subset(dataset, val_indices)

        train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True, pin_memory=True, num_workers=4)
        test_dataloader = DataLoader(val_set, batch_size=32, shuffle=True, pin_memory=True, num_workers=4)
        
        train_loss_list, val_loss_list = train(model, criterion, optimizer, train_dataloader, test_dataloader, early_stopper, num_epochs, trace_func)
        
        model.load_state_dict(torch.load("best_weights.pt"))

        train_score.at[i] = train_loss_list
        val_score.at[i] = val_loss_list
        unscaled_loss.at[i] = get_unscaled_loss(model, criterion, test_dataloader, dataset)
    
    os.remove("base_weights.pt")
    os.remove("best_weights.pt")
    
    return train_score, val_score, unscaled_loss

In [ ]:
dataset = RasterDataset(data_dir="../aug_dataset")
model = RasterNet().to(dml)

tr_score, val_score, unscaled_loss = cross_validator(model, dataset, k_fold=6, num_epochs=150, patience=15, learning_rate=0.001, verbose=True, trace_func=print)


In [ ]:

figure, axis = plt.subplots(3, 2)
for i in range(3):
    for j in range(2):
        axis[i][j].plot(tr_score[i*2+j], label="Training Loss", linewidth=1.5)
        axis[i][j].plot(val_score[i*2+j], label="Validation Loss", linewidth=1.5)
        axis[i][j].set_xlabel("Epochs")
        axis[i][j].set_ylabel("Loss")

handles, labels = axis[i][j].get_legend_handles_labels()
figure.legend(handles, labels, loc='lower center')

figure.tight_layout()
figure.show()

In [ ]:
unscaled_loss = pd.DataFrame(unscaled_loss.tolist(), columns=['avg','std'], index=unscaled_loss.index)
avg_loss = unscaled_loss['avg']
std_loss = unscaled_loss['std']

plt.errorbar(x=range(6), y=avg_loss, yerr=std_loss, fmt='o')
plt.xlabel("Fold")
plt.ylabel("TBV Loss")

In [ ]:
display(unscaled_loss)